# Chapter 2: Linear Algebra for Atomic Manipulation

## Learning Objectives
- Master rotation matrices around coordinate axes
- Understand the mathematics of arbitrary axis rotations (Rodrigues' formula)
- Learn affine transformations using 4×4 matrices
- Apply transformations to molecular structures
- Combine multiple transformations

---

## 2.1 Introduction to Transformation Matrices

In computational chemistry, we frequently need to:
- **Rotate** molecules to align bonds or compare structures
- **Translate** structures to center them or combine them
- **Reflect** structures to generate mirror images
- **Scale** coordinates (less common but useful)

All these operations can be expressed as **matrix operations**, making them computationally efficient and mathematically elegant.

### Why Matrices?

1. **Efficiency**: Matrix-vector multiplication is highly optimized in NumPy
2. **Composition**: Multiple transformations combine by matrix multiplication
3. **Invertibility**: We can undo transformations by computing matrix inverses
4. **Generality**: The same framework works for all linear transformations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import List, Tuple, Optional

# Import our Structure class from Chapter 1 (or redefine it)
class Atom:
    ELEMENTS = {
        'H':  {'Z': 1,  'mass': 1.008,   'color': 'white',  'radius': 0.31},
        'C':  {'Z': 6,  'mass': 12.011,  'color': 'gray',   'radius': 0.76},
        'N':  {'Z': 7,  'mass': 14.007,  'color': 'blue',   'radius': 0.71},
        'O':  {'Z': 8,  'mass': 15.999,  'color': 'red',    'radius': 0.66},
        'Si': {'Z': 14, 'mass': 28.086,  'color': 'gold',   'radius': 1.11},
        'Fe': {'Z': 26, 'mass': 55.845,  'color': 'orange', 'radius': 1.32},
        'Cu': {'Z': 29, 'mass': 63.546,  'color': 'brown',  'radius': 1.40},
        'Au': {'Z': 79, 'mass': 196.967, 'color': 'gold',   'radius': 1.36},
    }
    
    def __init__(self, symbol: str, position: np.ndarray):
        self.symbol = symbol
        self.position = np.array(position, dtype=float)
        if symbol in self.ELEMENTS:
            props = self.ELEMENTS[symbol]
            self.atomic_number = props['Z']
            self.mass = props['mass']
            self.color = props['color']
            self.covalent_radius = props['radius']
        else:
            self.atomic_number = 0
            self.mass = 1.0
            self.color = 'gray'
            self.covalent_radius = 1.0

class Structure:
    def __init__(self, name: str = "unnamed"):
        self.name = name
        self.atoms: List[Atom] = []
    
    def add_atom(self, symbol: str, position: np.ndarray) -> None:
        self.atoms.append(Atom(symbol, position))
    
    def __len__(self) -> int:
        return len(self.atoms)
    
    def get_positions(self) -> np.ndarray:
        return np.array([atom.position for atom in self.atoms])
    
    def set_positions(self, positions: np.ndarray) -> None:
        for atom, pos in zip(self.atoms, positions):
            atom.position = pos
    
    def get_symbols(self) -> List[str]:
        return [atom.symbol for atom in self.atoms]
    
    def get_center_of_mass(self) -> np.ndarray:
        total_mass = sum(atom.mass for atom in self.atoms)
        weighted_pos = sum(atom.mass * atom.position for atom in self.atoms)
        return weighted_pos / total_mass
    
    def copy(self) -> 'Structure':
        """Create a deep copy of the structure."""
        new_struct = Structure(self.name + "_copy")
        for atom in self.atoms:
            new_struct.add_atom(atom.symbol, atom.position.copy())
        return new_struct

print("Classes loaded successfully!")

## 2.2 Rotation Matrices Around Coordinate Axes

### Rotation Around the Z-axis

When we rotate a point $(x, y, z)$ around the z-axis by angle $\theta$:
- The z-coordinate stays the same
- The x and y coordinates rotate in the xy-plane

$$R_z(\theta) = \begin{pmatrix} \cos\theta & -\sin\theta & 0 \\ \sin\theta & \cos\theta & 0 \\ 0 & 0 & 1 \end{pmatrix}$$

### Rotation Around the X-axis

$$R_x(\theta) = \begin{pmatrix} 1 & 0 & 0 \\ 0 & \cos\theta & -\sin\theta \\ 0 & \sin\theta & \cos\theta \end{pmatrix}$$

### Rotation Around the Y-axis

$$R_y(\theta) = \begin{pmatrix} \cos\theta & 0 & \sin\theta \\ 0 & 1 & 0 \\ -\sin\theta & 0 & \cos\theta \end{pmatrix}$$

### Key Properties of Rotation Matrices

1. **Orthogonal**: $R^T R = I$ (transpose equals inverse)
2. **Determinant = 1**: Preserves orientation (no reflection)
3. **Preserve distances**: $|R\mathbf{v}| = |\mathbf{v}|$
4. **Preserve angles**: $(R\mathbf{v}_1) \cdot (R\mathbf{v}_2) = \mathbf{v}_1 \cdot \mathbf{v}_2$

In [ ]:
def rotation_matrix_x(theta: float) -> np.ndarray:
    """Create a rotation matrix for rotation around the x-axis.
    
    Args:
        theta: Rotation angle in radians
    
    Returns:
        3x3 rotation matrix
    """
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [1, 0,  0],
        [0, c, -s],
        [0, s,  c]
    ])

def rotation_matrix_y(theta: float) -> np.ndarray:
    """Create a rotation matrix for rotation around the y-axis."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [ c, 0, s],
        [ 0, 1, 0],
        [-s, 0, c]
    ])

def rotation_matrix_z(theta: float) -> np.ndarray:
    """Create a rotation matrix for rotation around the z-axis."""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([
        [c, -s, 0],
        [s,  c, 0],
        [0,  0, 1]
    ])

# Example: Rotate a point around the z-axis
point = np.array([1.0, 0.0, 0.0])
angle = np.pi / 2  # 90 degrees

R_z = rotation_matrix_z(angle)
rotated_point = R_z @ point

print(f"Original point: {point}")
print(f"After 90° rotation around z: {rotated_point}")
print(f"Expected: [0, 1, 0]")

# Verify orthogonality
print(f"\nR_z^T @ R_z = \n{np.round(R_z.T @ R_z, 10)}")
print(f"det(R_z) = {np.linalg.det(R_z):.6f}")

## 2.3 Visualizing Rotations

In [ ]:
def plot_rotation_demo(axis: str = 'z'):
    """Demonstrate rotation around a coordinate axis."""
    fig = plt.figure(figsize=(12, 4))
    
    # Create a simple "molecule" - a triangle in the xy-plane
    original_points = np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 0.5]
    ])
    
    angles = [0, np.pi/4, np.pi/2, np.pi]
    titles = ['0°', '45°', '90°', '180°']
    
    if axis == 'x':
        rot_func = rotation_matrix_x
    elif axis == 'y':
        rot_func = rotation_matrix_y
    else:
        rot_func = rotation_matrix_z
    
    for i, (angle, title) in enumerate(zip(angles, titles)):
        ax = fig.add_subplot(1, 4, i+1, projection='3d')
        
        R = rot_func(angle)
        rotated = (R @ original_points.T).T
        
        # Plot original in light color
        ax.scatter(*original_points.T, c='lightblue', s=100, alpha=0.5, label='Original')
        # Plot rotated in dark color
        ax.scatter(*rotated.T, c='blue', s=100, label='Rotated')
        
        # Draw edges
        for j in range(3):
            k = (j + 1) % 3
            ax.plot3D(*zip(rotated[j], rotated[k]), 'b-', alpha=0.7)
        
        # Draw rotation axis
        axis_line = np.array([[-1.5, 1.5], [0, 0], [0, 0]])
        if axis == 'x':
            ax.plot3D(*axis_line, 'r-', linewidth=2, label='Rotation axis')
        elif axis == 'y':
            ax.plot3D(axis_line[1], axis_line[0], axis_line[2], 'g-', linewidth=2)
        else:
            ax.plot3D(axis_line[1], axis_line[2], axis_line[0], 'purple', linewidth=2)
        
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-1.5, 1.5)
        ax.set_zlim(-1.5, 1.5)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(f'R_{axis}({title})')
    
    plt.tight_layout()
    plt.show()

print("Rotation around Z-axis:")
plot_rotation_demo('z')

print("Rotation around X-axis:")
plot_rotation_demo('x')

## 2.4 Rotation Around an Arbitrary Axis: Rodrigues' Formula

### The Problem

What if we need to rotate around an axis that isn't x, y, or z? For example:
- Rotate a molecule around a bond axis
- Align a molecule to a specific direction

### Rodrigues' Rotation Formula

To rotate by angle $\theta$ around a unit vector $\mathbf{k} = (k_x, k_y, k_z)$:

$$R = I + (\sin\theta)K + (1-\cos\theta)K^2$$

Where $K$ is the **skew-symmetric cross-product matrix**:

$$K = \begin{pmatrix} 0 & -k_z & k_y \\ k_z & 0 & -k_x \\ -k_y & k_x & 0 \end{pmatrix}$$

This matrix $K$ has the property that $K\mathbf{v} = \mathbf{k} \times \mathbf{v}$ (cross product).

### Explicit Form

$$R = \begin{pmatrix}
\cos\theta + k_x^2(1-\cos\theta) & k_x k_y(1-\cos\theta) - k_z\sin\theta & k_x k_z(1-\cos\theta) + k_y\sin\theta \\
k_y k_x(1-\cos\theta) + k_z\sin\theta & \cos\theta + k_y^2(1-\cos\theta) & k_y k_z(1-\cos\theta) - k_x\sin\theta \\
k_z k_x(1-\cos\theta) - k_y\sin\theta & k_z k_y(1-\cos\theta) + k_x\sin\theta & \cos\theta + k_z^2(1-\cos\theta)
\end{pmatrix}$$

In [ ]:
def rotation_matrix_axis_angle(axis: np.ndarray, theta: float) -> np.ndarray:
    """Create a rotation matrix for rotation around an arbitrary axis.
    
    Uses Rodrigues' rotation formula.
    
    Args:
        axis: The rotation axis (will be normalized)
        theta: Rotation angle in radians
    
    Returns:
        3x3 rotation matrix
    """
    # Normalize the axis
    k = axis / np.linalg.norm(axis)
    kx, ky, kz = k
    
    # Create the skew-symmetric matrix K
    K = np.array([
        [0, -kz, ky],
        [kz, 0, -kx],
        [-ky, kx, 0]
    ])
    
    # Rodrigues' formula
    I = np.eye(3)
    R = I + np.sin(theta) * K + (1 - np.cos(theta)) * (K @ K)
    
    return R

# Alternative implementation using explicit formula
def rotation_matrix_axis_angle_explicit(axis: np.ndarray, theta: float) -> np.ndarray:
    """Alternative implementation using the explicit formula."""
    k = axis / np.linalg.norm(axis)
    kx, ky, kz = k
    c, s = np.cos(theta), np.sin(theta)
    t = 1 - c
    
    return np.array([
        [c + kx*kx*t,      kx*ky*t - kz*s,  kx*kz*t + ky*s],
        [ky*kx*t + kz*s,   c + ky*ky*t,     ky*kz*t - kx*s],
        [kz*kx*t - ky*s,   kz*ky*t + kx*s,  c + kz*kz*t]
    ])

# Verify: rotation around z-axis should match our earlier function
z_axis = np.array([0, 0, 1])
angle = np.pi / 3  # 60 degrees

R1 = rotation_matrix_z(angle)
R2 = rotation_matrix_axis_angle(z_axis, angle)

print("Verification: R_z using two methods")
print(f"Standard R_z:\n{np.round(R1, 6)}")
print(f"\nRodrigues R_z:\n{np.round(R2, 6)}")
print(f"\nDifference: {np.max(np.abs(R1 - R2)):.2e}")

# Example: Rotate around the [1,1,1] axis (body diagonal)
diagonal_axis = np.array([1, 1, 1])
R_diag = rotation_matrix_axis_angle(diagonal_axis, 2*np.pi/3)  # 120 degrees

# This rotation should cyclically permute (x,y,z) → (y,z,x)
test_point = np.array([1, 2, 3])
rotated = R_diag @ test_point

print(f"\n120° rotation around [1,1,1]:")
print(f"Original: {test_point}")
print(f"Rotated:  {np.round(rotated, 6)}")
print(f"Expected: [2, 3, 1] (cyclic permutation)")

## 2.5 Applying Rotations to Structures

Now let's apply rotations to molecular structures. The key insight is that we can rotate all atomic positions simultaneously using matrix multiplication.

In [ ]:
def rotate_structure(structure: Structure, rotation_matrix: np.ndarray, 
                     center: Optional[np.ndarray] = None) -> Structure:
    """Rotate a structure around a center point.
    
    Args:
        structure: The structure to rotate
        rotation_matrix: 3x3 rotation matrix
        center: Center of rotation (default: center of mass)
    
    Returns:
        A new rotated structure
    """
    new_structure = structure.copy()
    
    # Default to center of mass
    if center is None:
        center = structure.get_center_of_mass()
    
    # Get positions, translate to origin, rotate, translate back
    positions = new_structure.get_positions()
    centered = positions - center
    rotated = (rotation_matrix @ centered.T).T
    final = rotated + center
    
    new_structure.set_positions(final)
    return new_structure

def rotate_around_bond(structure: Structure, atom1_idx: int, atom2_idx: int, 
                       angle: float) -> Structure:
    """Rotate a structure around a bond axis.
    
    Args:
        structure: The structure to rotate
        atom1_idx: Index of first atom defining the bond
        atom2_idx: Index of second atom defining the bond
        angle: Rotation angle in degrees
    
    Returns:
        A new rotated structure
    """
    # Get the bond axis
    pos1 = structure.atoms[atom1_idx].position
    pos2 = structure.atoms[atom2_idx].position
    axis = pos2 - pos1
    
    # Create rotation matrix
    R = rotation_matrix_axis_angle(axis, np.radians(angle))
    
    # Rotate around atom1
    return rotate_structure(structure, R, center=pos1)

# Build a water molecule
water = Structure("H2O")
water.add_atom('O', [0.0, 0.0, 0.0])
water.add_atom('H', [0.96, 0.0, 0.0])
water.add_atom('H', [0.96 * np.cos(np.radians(104.5)), 
                     0.96 * np.sin(np.radians(104.5)), 0.0])

# Rotate by 90 degrees around z-axis
R_z90 = rotation_matrix_z(np.pi/2)
water_rotated = rotate_structure(water, R_z90)

# Visualize
fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(121, projection='3d')
for atom in water.atoms:
    ax1.scatter(*atom.position, c=atom.color, s=300, edgecolors='black')
ax1.set_title('Original Water')
ax1.set_xlabel('X'); ax1.set_ylabel('Y'); ax1.set_zlabel('Z')

ax2 = fig.add_subplot(122, projection='3d')
for atom in water_rotated.atoms:
    ax2.scatter(*atom.position, c=atom.color, s=300, edgecolors='black')
ax2.set_title('Water Rotated 90° around Z')
ax2.set_xlabel('X'); ax2.set_ylabel('Y'); ax2.set_zlabel('Z')

# Set same limits
for ax in [ax1, ax2]:
    ax.set_xlim(-1.5, 1.5); ax.set_ylim(-1.5, 1.5); ax.set_zlim(-1.5, 1.5)

plt.tight_layout()
plt.show()

## 2.6 Affine Transformations with 4×4 Matrices

### The Limitation of 3×3 Matrices

A 3×3 matrix can represent rotations, reflections, and scaling, but NOT translations. To include translations, we use **homogeneous coordinates** and 4×4 matrices.

### Homogeneous Coordinates

A 3D point $(x, y, z)$ is represented as $(x, y, z, 1)$ in homogeneous coordinates.

### General Affine Transformation

$$\begin{pmatrix} x' \\ y' \\ z' \\ 1 \end{pmatrix} = 
\begin{pmatrix} 
R_{11} & R_{12} & R_{13} & t_x \\
R_{21} & R_{22} & R_{23} & t_y \\
R_{31} & R_{32} & R_{33} & t_z \\
0 & 0 & 0 & 1
\end{pmatrix}
\begin{pmatrix} x \\ y \\ z \\ 1 \end{pmatrix}$$

Where:
- The upper-left 3×3 block is the rotation/scaling matrix
- $(t_x, t_y, t_z)$ is the translation vector
- The bottom row is always $(0, 0, 0, 1)$

### Advantages

1. **Unified representation**: Rotations and translations in one matrix
2. **Easy composition**: Multiple transformations combine by matrix multiplication
3. **Invertible**: Easy to compute inverse transformation

In [ ]:
def translation_matrix(t: np.ndarray) -> np.ndarray:
    """Create a 4x4 translation matrix.
    
    Args:
        t: Translation vector [tx, ty, tz]
    
    Returns:
        4x4 transformation matrix
    """
    T = np.eye(4)
    T[:3, 3] = t
    return T

def rotation_matrix_4x4(R: np.ndarray) -> np.ndarray:
    """Convert a 3x3 rotation matrix to 4x4 affine form."""
    T = np.eye(4)
    T[:3, :3] = R
    return T

def scaling_matrix(s: np.ndarray) -> np.ndarray:
    """Create a 4x4 scaling matrix.
    
    Args:
        s: Scale factors [sx, sy, sz] or scalar for uniform scaling
    """
    if np.isscalar(s):
        s = np.array([s, s, s])
    S = np.eye(4)
    S[0, 0] = s[0]
    S[1, 1] = s[1]
    S[2, 2] = s[2]
    return S

def apply_transform(positions: np.ndarray, T: np.ndarray) -> np.ndarray:
    """Apply a 4x4 transformation matrix to a set of positions.
    
    Args:
        positions: Nx3 array of positions
        T: 4x4 transformation matrix
    
    Returns:
        Nx3 array of transformed positions
    """
    n = len(positions)
    # Convert to homogeneous coordinates
    homogeneous = np.hstack([positions, np.ones((n, 1))])
    # Apply transformation
    transformed = (T @ homogeneous.T).T
    # Convert back to 3D
    return transformed[:, :3]

# Example: combine rotation and translation
# First rotate 90° around z, then translate by [1, 2, 0]
R = rotation_matrix_4x4(rotation_matrix_z(np.pi/2))
T = translation_matrix([1, 2, 0])

# Combined transformation: FIRST rotation, THEN translation
# Remember: matrix multiplication is right-to-left
combined = T @ R

point = np.array([[1, 0, 0]])
result = apply_transform(point, combined)

print("Combined rotation + translation:")
print(f"Original point: {point[0]}")
print(f"After rotation: {apply_transform(point, R)[0]}")
print(f"After translation: {result[0]}")

print("\nCombined 4x4 matrix:")
print(np.round(combined, 4))

## 2.7 Composing Transformations: Order Matters!

**Critical concept**: Matrix multiplication is not commutative. The order of transformations matters!

$$T_1 \cdot T_2 \neq T_2 \cdot T_1$$

Transformations are applied **right to left**:
- $T_{final} = T_3 \cdot T_2 \cdot T_1$ means: apply $T_1$ first, then $T_2$, then $T_3$

In [ ]:
# Demonstrate that order matters
R90_z = rotation_matrix_4x4(rotation_matrix_z(np.pi/2))
T_x = translation_matrix([2, 0, 0])

# Order 1: Rotate, then translate
RT = T_x @ R90_z

# Order 2: Translate, then rotate
TR = R90_z @ T_x

point = np.array([[1, 0, 0]])

result_RT = apply_transform(point, RT)
result_TR = apply_transform(point, TR)

print("Original point:", point[0])
print(f"\nRotate then translate (RT): {result_RT[0]}")
print("  Step 1 - Rotate 90°: [1,0,0] → [0,1,0]")
print("  Step 2 - Translate [2,0,0]: [0,1,0] → [2,1,0]")

print(f"\nTranslate then rotate (TR): {result_TR[0]}")
print("  Step 1 - Translate [2,0,0]: [1,0,0] → [3,0,0]")
print("  Step 2 - Rotate 90°: [3,0,0] → [0,3,0]")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for ax, title, result in zip(axes, ['Rotate → Translate', 'Translate → Rotate'], 
                               [result_RT, result_TR]):
    ax.scatter(*point[0, :2], c='blue', s=200, label='Original', zorder=5)
    ax.scatter(*result[0, :2], c='red', s=200, label='Final', zorder=5)
    ax.arrow(0, 0, point[0, 0]*0.9, point[0, 1]*0.9, head_width=0.1, color='blue', alpha=0.5)
    ax.arrow(0, 0, result[0, 0]*0.9, result[0, 1]*0.9, head_width=0.1, color='red', alpha=0.5)
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
    ax.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
    ax.set_xlim(-1, 4)
    ax.set_ylim(-1, 4)
    ax.set_aspect('equal')
    ax.legend()
    ax.set_title(title)
    ax.set_xlabel('X'); ax.set_ylabel('Y')

plt.tight_layout()
plt.show()

## 2.8 Rotating Around a Point (Not the Origin)

To rotate around a point $\mathbf{p}$ that isn't the origin:

1. Translate so that $\mathbf{p}$ moves to the origin: $T_{-p}$
2. Apply the rotation: $R$
3. Translate back: $T_{+p}$

$$T_{total} = T_{+p} \cdot R \cdot T_{-p}$$

In [ ]:
def rotation_around_point(R: np.ndarray, center: np.ndarray) -> np.ndarray:
    """Create a 4x4 matrix for rotation around an arbitrary point.
    
    Args:
        R: 3x3 rotation matrix
        center: Point to rotate around [x, y, z]
    
    Returns:
        4x4 transformation matrix
    """
    center = np.array(center)
    T_neg = translation_matrix(-center)
    T_pos = translation_matrix(center)
    R_4x4 = rotation_matrix_4x4(R)
    
    return T_pos @ R_4x4 @ T_neg

def rotation_around_axis_point(axis: np.ndarray, point: np.ndarray, 
                                angle: float) -> np.ndarray:
    """Create transformation for rotation around an axis through a point.
    
    Args:
        axis: Direction of rotation axis
        point: A point on the rotation axis
        angle: Rotation angle in radians
    
    Returns:
        4x4 transformation matrix
    """
    R = rotation_matrix_axis_angle(axis, angle)
    return rotation_around_point(R, point)

# Example: Rotate a square around one of its corners
square = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 0]
])

# Rotate around the corner at [0,0,0]
R_45 = rotation_matrix_z(np.pi/4)  # 45 degrees
T_corner = rotation_around_point(R_45, [0, 0, 0])
square_rot_corner = apply_transform(square, T_corner)

# Rotate around the center [0.5, 0.5, 0]
T_center = rotation_around_point(R_45, [0.5, 0.5, 0])
square_rot_center = apply_transform(square, T_center)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for ax, rotated, title, center in zip(
    axes, 
    [square_rot_corner, square_rot_center],
    ['Rotate around corner [0,0,0]', 'Rotate around center [0.5,0.5,0]'],
    [[0, 0], [0.5, 0.5]]):
    
    # Original
    sq = np.vstack([square, square[0]])  # Close the square
    ax.plot(sq[:, 0], sq[:, 1], 'b-', linewidth=2, label='Original')
    
    # Rotated
    rot = np.vstack([rotated, rotated[0]])
    ax.plot(rot[:, 0], rot[:, 1], 'r--', linewidth=2, label='Rotated 45°')
    
    # Rotation center
    ax.scatter(*center, c='green', s=100, marker='x', linewidths=3, 
               label='Rotation center', zorder=10)
    
    ax.set_xlim(-1.5, 2)
    ax.set_ylim(-1, 2)
    ax.set_aspect('equal')
    ax.legend()
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2.9 Reflection (Mirror) Operations

Reflections are important for:
- Creating mirror-image molecules (enantiomers)
- Symmetry operations in crystals

### Reflection Through Coordinate Planes

**Reflection through the xy-plane** (z → -z):
$$\sigma_{xy} = \begin{pmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & -1 \end{pmatrix}$$

**Reflection through the xz-plane** (y → -y):
$$\sigma_{xz} = \begin{pmatrix} 1 & 0 & 0 \\ 0 & -1 & 0 \\ 0 & 0 & 1 \end{pmatrix}$$

**Reflection through the yz-plane** (x → -x):
$$\sigma_{yz} = \begin{pmatrix} -1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{pmatrix}$$

### Reflection Through an Arbitrary Plane

For a plane with normal vector $\mathbf{n}$ passing through the origin:

$$\sigma = I - 2\mathbf{n}\mathbf{n}^T$$

This is the **Householder reflection matrix**.

In [ ]:
def reflection_matrix(normal: np.ndarray) -> np.ndarray:
    """Create a reflection matrix for reflection through a plane.
    
    The plane passes through the origin with the given normal vector.
    
    Args:
        normal: Normal vector to the reflection plane
    
    Returns:
        3x3 reflection matrix
    """
    n = normal / np.linalg.norm(normal)
    # Householder reflection: I - 2 * n * n^T
    return np.eye(3) - 2 * np.outer(n, n)

# Predefined reflection matrices
sigma_xy = reflection_matrix([0, 0, 1])  # Reflect through xy-plane
sigma_xz = reflection_matrix([0, 1, 0])  # Reflect through xz-plane
sigma_yz = reflection_matrix([1, 0, 0])  # Reflect through yz-plane

print("σ_xy (reflection through xy-plane):")
print(sigma_xy)

# Verify: reflecting [1,2,3] through xy-plane should give [1,2,-3]
point = np.array([1, 2, 3])
reflected = sigma_xy @ point
print(f"\n{point} reflected through xy-plane: {reflected}")

# Key property: det(reflection) = -1 (orientation reverses)
print(f"\ndet(σ_xy) = {np.linalg.det(sigma_xy):.1f}")

# Reflection is its own inverse: σ² = I
print(f"σ_xy @ σ_xy = I? {np.allclose(sigma_xy @ sigma_xy, np.eye(3))}")

## 2.10 Inversion

**Inversion** through a point (usually the origin) sends $(x, y, z) \to (-x, -y, -z)$:

$$i = \begin{pmatrix} -1 & 0 & 0 \\ 0 & -1 & 0 \\ 0 & 0 & -1 \end{pmatrix} = -I$$

This is equivalent to reflection through all three coordinate planes simultaneously, or a 180° rotation combined with reflection.

In [ ]:
def inversion_matrix() -> np.ndarray:
    """Create the inversion matrix."""
    return -np.eye(3)

inv = inversion_matrix()

# Create NH3 and its inverted form
nh3 = Structure("NH3")
nh3.add_atom('N', [0, 0, 0])
bond = 1.02
for i in range(3):
    phi = i * 2 * np.pi / 3
    nh3.add_atom('H', [bond * 0.8 * np.cos(phi), 
                       bond * 0.8 * np.sin(phi), 
                       -0.4])

# Invert
nh3_inv = nh3.copy()
nh3_inv.name = "NH3_inverted"
positions = nh3_inv.get_positions()
nh3_inv.set_positions((inv @ positions.T).T)

# Visualize
fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(121, projection='3d')
for atom in nh3.atoms:
    ax1.scatter(*atom.position, c=atom.color, s=300, edgecolors='black')
ax1.set_title('NH3 Original')

ax2 = fig.add_subplot(122, projection='3d')
for atom in nh3_inv.atoms:
    ax2.scatter(*atom.position, c=atom.color, s=300, edgecolors='black')
ax2.set_title('NH3 Inverted')

for ax in [ax1, ax2]:
    ax.set_xlim(-1.5, 1.5); ax.set_ylim(-1.5, 1.5); ax.set_zlim(-1.5, 1.5)
    ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')

plt.tight_layout()
plt.show()

## 2.11 Aligning Vectors: The Alignment Problem

A common task: find a rotation that aligns one vector to another.

Given vectors $\mathbf{a}$ and $\mathbf{b}$, find rotation matrix $R$ such that $R\mathbf{a} = \mathbf{b}$.

In [ ]:
def rotation_align_vectors(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Find rotation matrix that aligns vector a to vector b.
    
    Args:
        a: Source vector (will be rotated)
        b: Target vector (destination)
    
    Returns:
        3x3 rotation matrix R such that R @ a/|a| = b/|b|
    """
    # Normalize inputs
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    
    # Check if vectors are parallel
    if np.allclose(a, b):
        return np.eye(3)
    if np.allclose(a, -b):
        # 180° rotation - need to find perpendicular axis
        # Use the axis least aligned with a
        perp = np.array([1, 0, 0])
        if abs(np.dot(a, perp)) > 0.9:
            perp = np.array([0, 1, 0])
        axis = np.cross(a, perp)
        return rotation_matrix_axis_angle(axis, np.pi)
    
    # General case: use Rodrigues formula
    # Rotation axis is perpendicular to both vectors
    axis = np.cross(a, b)
    
    # Rotation angle
    cos_angle = np.dot(a, b)
    angle = np.arccos(np.clip(cos_angle, -1, 1))
    
    return rotation_matrix_axis_angle(axis, angle)

# Example: Align a molecule's bond to the z-axis
# Start with bond in arbitrary direction
bond_vector = np.array([1, 1, 0])
z_axis = np.array([0, 0, 1])

R_align = rotation_align_vectors(bond_vector, z_axis)

print(f"Original bond direction: {bond_vector}")
print(f"After alignment: {np.round(R_align @ bond_vector / np.linalg.norm(bond_vector), 6)}")
print(f"Target direction: {z_axis}")

# Apply to a full molecule - align the O-H bond of water to z-axis
water = Structure("H2O")
water.add_atom('O', [0.0, 0.0, 0.0])
water.add_atom('H', [0.96, 0.0, 0.0])
water.add_atom('H', [0.96 * np.cos(np.radians(104.5)), 
                     0.96 * np.sin(np.radians(104.5)), 0.0])

# Get O-H1 bond vector
O_pos = water.atoms[0].position
H1_pos = water.atoms[1].position
OH_bond = H1_pos - O_pos

# Align to z-axis
R = rotation_align_vectors(OH_bond, z_axis)
water_aligned = rotate_structure(water, R, center=O_pos)

print(f"\nWater molecule positions after aligning O-H1 to z-axis:")
for i, atom in enumerate(water_aligned.atoms):
    print(f"{atom.symbol}: {np.round(atom.position, 4)}")

## 2.12 Euler Angles

Euler angles describe any rotation as three successive rotations around coordinate axes. Different conventions exist; the most common is **ZYZ** or **ZXZ**.

### ZYZ Convention

Rotation by angles $(\alpha, \beta, \gamma)$:

$$R = R_z(\alpha) \cdot R_y(\beta) \cdot R_z(\gamma)$$

### Properties
- 3 degrees of freedom (matches the 3 DOF of rotation in 3D)
- Gimbal lock: singularity when $\beta = 0$ or $\pi$

In [ ]:
def euler_to_rotation_matrix(alpha: float, beta: float, gamma: float, 
                              convention: str = 'ZYZ') -> np.ndarray:
    """Convert Euler angles to rotation matrix.
    
    Args:
        alpha, beta, gamma: Euler angles in radians
        convention: Rotation sequence ('ZYZ', 'ZXZ', 'XYZ', etc.)
    
    Returns:
        3x3 rotation matrix
    """
    rot_funcs = {
        'X': rotation_matrix_x,
        'Y': rotation_matrix_y,
        'Z': rotation_matrix_z
    }
    
    R1 = rot_funcs[convention[0]](alpha)
    R2 = rot_funcs[convention[1]](beta)
    R3 = rot_funcs[convention[2]](gamma)
    
    return R1 @ R2 @ R3

def rotation_matrix_to_euler(R: np.ndarray, convention: str = 'ZYZ') -> Tuple[float, float, float]:
    """Convert rotation matrix to Euler angles (ZYZ convention).
    
    Returns:
        Tuple of (alpha, beta, gamma) in radians
    """
    if convention != 'ZYZ':
        raise NotImplementedError("Only ZYZ convention implemented")
    
    # Handle gimbal lock cases
    if np.isclose(R[2, 2], 1.0):
        # beta = 0 (gimbal lock)
        beta = 0
        alpha = np.arctan2(R[0, 1], R[0, 0])
        gamma = 0
    elif np.isclose(R[2, 2], -1.0):
        # beta = pi (gimbal lock)
        beta = np.pi
        alpha = np.arctan2(R[0, 1], -R[0, 0])
        gamma = 0
    else:
        beta = np.arccos(R[2, 2])
        alpha = np.arctan2(R[1, 2], R[0, 2])
        gamma = np.arctan2(R[2, 1], -R[2, 0])
    
    return alpha, beta, gamma

# Example
alpha, beta, gamma = np.radians(30), np.radians(45), np.radians(60)
R = euler_to_rotation_matrix(alpha, beta, gamma)

print(f"Euler angles: α={np.degrees(alpha):.1f}°, β={np.degrees(beta):.1f}°, γ={np.degrees(gamma):.1f}°")
print(f"\nRotation matrix:")
print(np.round(R, 4))

# Convert back
a2, b2, g2 = rotation_matrix_to_euler(R)
print(f"\nRecovered angles: α={np.degrees(a2):.1f}°, β={np.degrees(b2):.1f}°, γ={np.degrees(g2):.1f}°")

---

## Practice Exercises

### Exercise 2.1: Verify Rotation Matrix Properties
Write a function that verifies all properties of a rotation matrix (orthogonality, determinant = 1, preserves lengths).

In [ ]:
# YOUR CODE HERE
def verify_rotation_matrix(R: np.ndarray, tol: float = 1e-10) -> dict:
    """Verify that a matrix is a valid rotation matrix.
    
    Returns:
        Dictionary with verification results
    """
    # TODO: Check orthogonality (R^T @ R = I)
    # TODO: Check determinant = 1
    # TODO: Check that it preserves vector lengths
    pass

# Test with various matrices

### Exercise 2.2: Chain of Rotations
Create a function that applies a sequence of rotations defined by (axis, angle) pairs.

In [ ]:
# YOUR CODE HERE
def combined_rotation(rotations: List[Tuple[str, float]]) -> np.ndarray:
    """Create a combined rotation matrix from a sequence of rotations.
    
    Args:
        rotations: List of (axis, angle_in_degrees) tuples
                   axis can be 'x', 'y', 'z', or a 3D vector
    
    Example:
        combined_rotation([('z', 90), ('x', 45), ('y', 30)])
    """
    # TODO: Implement
    pass

### Exercise 2.3: Molecular Alignment
Write a function that aligns any bond in a molecule to point along the z-axis, with a specified atom at the origin.

In [ ]:
# YOUR CODE HERE
def align_bond_to_z(structure: Structure, atom1_idx: int, atom2_idx: int) -> Structure:
    """Align a bond to the z-axis with atom1 at the origin.
    
    After transformation:
    - atom1 is at the origin
    - atom2 is on the positive z-axis
    """
    # TODO: Implement
    pass

### Exercise 2.4: Improper Rotation (Roto-Reflection)
An improper rotation $S_n$ is a rotation by $360°/n$ followed by reflection through a plane perpendicular to the rotation axis. Implement this operation.

In [ ]:
# YOUR CODE HERE
def improper_rotation_matrix(axis: np.ndarray, n: int) -> np.ndarray:
    """Create an improper rotation (roto-reflection) matrix.
    
    S_n = rotation by 2π/n followed by reflection perpendicular to axis
    
    Args:
        axis: Rotation axis (will also be normal to reflection plane)
        n: Order of the improper rotation
    
    Returns:
        3x3 transformation matrix
    """
    # TODO: Implement
    pass

### Exercise 2.5: Rotation Interpolation (SLERP)
Implement Spherical Linear Interpolation between two rotations for smooth animation.

In [ ]:
# YOUR CODE HERE
def interpolate_rotations(R1: np.ndarray, R2: np.ndarray, t: float) -> np.ndarray:
    """Interpolate between two rotation matrices.
    
    Args:
        R1: Starting rotation matrix
        R2: Ending rotation matrix
        t: Interpolation parameter (0 = R1, 1 = R2)
    
    Returns:
        Interpolated rotation matrix
    """
    # Hint: Use logarithm of rotation, interpolate, then exponentiate
    # Or use quaternions for cleaner implementation
    pass

---

## Key Takeaways

1. **Rotation matrices** are orthogonal with determinant +1
2. **Rodrigues' formula** allows rotation around any axis
3. **4×4 affine matrices** unify rotations and translations
4. **Order matters**: transformations compose right-to-left
5. **To rotate around a point**: translate to origin, rotate, translate back
6. **Reflections** have determinant -1 and are self-inverse

## Next Chapter Preview

In Chapter 3, we'll explore **Symmetry Operations and Point Groups**:
- Classification of symmetry elements
- Point groups in chemistry
- Applying symmetry to generate equivalent atoms
- Character tables (brief introduction)